In [1]:
import heapq
import numpy as np
from PIL import Image
import os
import random

def heuristic(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def astar(array, start, goal):
    neighbors = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}
    oheap = []

    heapq.heappush(oheap, (fscore[start], start))

    while oheap:
        current = heapq.heappop(oheap)[1]

        if current == goal:
            data = []
            while current in came_from:
                data.append(current)
                current = came_from[current]
            return data[::-1]

        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j
            tentative_g_score = gscore[current] + heuristic(current, neighbor)
            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:
                    if array[neighbor[0]][neighbor[1]] == 1:
                        continue
                else:
                    continue
            else:
                continue

            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, 0):
                continue

            if tentative_g_score < gscore.get(neighbor, 0) or neighbor not in [i[1] for i in oheap]:
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))

    return False



import numpy as np
import os
import random
from PIL import Image

def generate_trajectory_data(map_dir, save_path, num_samples=1):
    trajectory_data = []
    maps = os.listdir(map_dir)

    for map_name in maps:
        map_path = os.path.join(map_dir, map_name)
        img = Image.open(map_path).convert('L')
        array = np.array(img)
        array = np.where(array > 128, 0, 1)  # Assuming white is traversable (0) and black is obstacle (1)

        # Find all traversable points in the maze
        traversable_points = np.argwhere(array == 0)

        samples_generated = 0
        while samples_generated < num_samples:
            # Randomly select start and goal from traversable points
            start = tuple(traversable_points[random.randint(0, len(traversable_points) - 1)])
            goal = tuple(traversable_points[random.randint(0, len(traversable_points) - 1)])

            path = astar(array, start, goal)
            if path:
                trajectory_data.append({
                    'map_name': map_name,
                    'start': start,
                    'goal': goal,
                    'trajectory': path
                })
                samples_generated += 1

    np.save(save_path, trajectory_data)
    print(f'Saved {len(trajectory_data)} trajectories to {save_path}')



# Generate trajectory data
generate_trajectory_data('maze_maps2', 'trajectories_2.npy', num_samples=1)



Saved 10 trajectories to trajectories_2.npy
